In [6]:
import pandas as pd

In [7]:
import mysql.connector as msql
from mysql.connector import Error
dataset = []
try:
    conn = msql.connect(host='localhost', database='DMHW1', user='root', password='TP6XU4JO3@@')
    if conn.is_connected():
        mycursor = conn.cursor()
        mycursor.execute("select database();")
        record = mycursor.fetchone()
        print("You're connected to database: ", record)
        mycursor.execute("""select 
concat(d.fname,IFNULL(d.mi,''),d.lname) as 'customer_name',
c.product_name
from DMHW1.sales_fact_1998 a join DMHW1.time_by_day b 
on a.time_id = b.time_id 
join DMHW1.product c 
on a.product_id = c.product_id 
join DMHW1.customer d
on a.customer_id = d.customer_id
union 
select  
concat(d.fname,IFNULL(d.mi,''),d.lname) as 'customer_name',
c.product_name
from DMHW1.sales_fact_dec_1998 a join DMHW1.time_by_day b 
on a.time_id = b.time_id 
join DMHW1.product c 
on a.product_id = c.product_id
join DMHW1.customer d
on a.customer_id = d.customer_id
group by customer_name, product_name limit 3300
                        """)
        print("data is fetched....")
        #loop through the data fetched
        results = mycursor.fetchall()
        a = dict()
        for item in results:
            row = list(item)
            for i in range(len(row)):
                if type(row[i]) is not str:
                    row[i] = str(row[i])
                if i == 0:
                    if row[i] not in a.keys():
                        a[row[i]] = []
                elif i != 0:
                    #print(row[i])
                    if row[i-1] in a.keys(): 
                        ls = a[row[i-1]]
                        ls.append(row[i])
                        a[row[i-1]] = ls
        for key,value in a.items():
            #print(value)
            dataset.append(value)
            #print(value)
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('dmhw1',)
data is fetched....


In [8]:
#from apyori import apriori
from mlxtend.frequent_patterns import fpmax, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [9]:
frequent_sets = fpgrowth(df, min_support=0.0001,use_colnames=True) 


In [10]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_sets, metric='confidence', min_threshold=0.9)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x:len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x:len(x))
#rules[(rules["antecedent_len"] >= 2)& (rules["consequent_len"] >=2)].sort_values(by=['support','confidence'], ascending=False).to_csv('has_promotion.csv')
rules[(rules["antecedent_len"] >= 1)& (rules["consequent_len"] >=1)].sort_values(by=['support','confidence'], ascending=False).to_csv('A_year_association_fp_v2.csv')
